In [1]:
from memory import MemoryBuffer
from policies import PPOTrainer
from utils import gaussian_likelihood, reshape_train_var
import tensorflow as tf
from tensorflow.layers import dense, conv2d, max_pooling2d, flatten
import numpy as np
import time
import gym

In [2]:
class IMTrainer():
    def __init__(self, in_op, out_op, v_out_op, act_type='discrete', sess=None):
        """
        Create a wrapper for RL networks for easy training.
        Args:
            in_op (tf.Placeholder): Observation input to architecture
            out_op (tf.Variable): Action output of architecture
            act_type (string): 'discrete' for a discrete actions space or 'continuous'
                               for a continuous actions space
            sess (tf.Session): A session if you would like to use a custom session,
                               if left none it will be automatically created
        """

        if not sess:
            self.renew_sess()
        
        self.in_op = in_op
        self.out_op = out_op
        self.v_out_op = v_out_op
        
        if act_type in ('discrete', 'd'):
            self.train = self._create_discrete_trainer()
            self.act_type = 'discrete'
        elif act_type in ('continuous', 'c'):
            self.train = self._create_continuous_trainer()
            self.act_type = 'continuous'
        else:
            raise TypeError('act_type must be \'discrete\' or \'continuous\'')
        
    def renew_sess(self):
        """
        Starts a new internal Tensorflow session
        """
        self.sess = tf.Session()
        
    def end_sess(self):
        """
        Ends the internal Tensorflow session if it exists
        """
        if self.sess:
            self.sess.close()
            
    def _create_ICM(self):
        feature_dim = 12
        r_i_scale = 3
        
        # Create placeholder
        self.next_obs_holders = tf.placeholder(tf.float32, shape=self.in_op.shape)
        
        # Observation feature encoder
        with tf.variable_scope('feature_encoder'):
            self.f_obs = dense(self.in_op, feature_dim, activation=tf.nn.tanh, name='fe_dense')
            
        with tf.variable_scope('feature_encoder', reuse=True):
            self.f_obs_next = dense(self.next_obs_holders, feature_dim, activation=tf.nn.tanh, name='fe_dense')
            
        # State predictor forward model
        self.state_act_pair = tf.concat([self.act_holders, self.f_obs], axis=1)
        self.sp_dense = dense(self.state_act_pair, 32, activation=tf.nn.tanh)
        self.f_obs_next_hat = dense(self.sp_dense, feature_dim, activation=tf.nn.tanh)
        
        # Inverse model (predicting action)
        self.state_state_pair = tf.concat([self.f_obs, self.f_obs_next], axis=1)
        self.act_hat = dense(self.state_state_pair, self.out_op.shape[1])
        
        # Calculating intrinsic reward
        self.obs_diff = self.f_obs_next_hat - self.f_obs_next
        self.r_i = r_i_scale * tf.reduce_sum(self.obs_diff ** 2, axis=1)
        
        # Calculating losses
        self.pre_loss_i = tf.reduce_sum((self.act_hat - self.act_holders) ** 2, axis=1)
        self.pre_loss_f = tf.reduce_sum(self.obs_diff ** 2, axis=1)
        
        self.loss_i = 0.5 * self.pre_loss_i
        self.loss_f = 0.5 * self.pre_loss_f
        
    def _create_discrete_trainer(self, optimizer=tf.train.AdamOptimizer()):
        """
        Creates a function for vanilla policy training with a discrete action space
        """
#         self.act_holders = tf.placeholder(tf.int32, shape=[None])
#         self.reward_holders = tf.placeholder(tf.float32, shape=[None])
        
#         self.act_masks = tf.one_hot(self.act_holders, self.out_op.shape[1].value, dtype=tf.float32)
#         self.log_probs = tf.log(self.out_op)
        
#         self.advantages = self.reward_holders - tf.squeeze(self.v_out_op)
        
#         self.resp_acts = tf.reduce_sum(self.act_masks *  self.log_probs, axis=1)
#         self.loss = -tf.reduce_mean(self.resp_acts * self.advantages)
        
#         self.optimizer = optimizer
#         self.actor_update = self.optimizer.minimize(self.loss)
        
#         with tf.control_dependencies([self.actor_update]):
#             self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - tf.squeeze(self.v_out_op)))
#             self.value_update = self.optimizer.minimize(self.value_loss)
        
#         update_func = lambda train_data: self.sess.run([self.actor_update, self.value_update], 
#                                                        feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
#                                                             self.act_holders: reshape_train_var(train_data[:, 1]),
#                                                             self.reward_holders: train_data[:, 2]})
        
#         self.sess.run(tf.global_variables_initializer())
        
#         return update_func
        
    def _create_continuous_trainer(self, optimizer=tf.train.AdamOptimizer()):
        """
        Creates a function for vanilla policy training with a continuous action space
        """
        self.act_holders = tf.placeholder(tf.float32, shape=[None, self.out_op.shape[1].value])
        self.reward_holders = tf.placeholder(tf.float32, shape=[None])
        
        self.std = tf.Variable(0.5 * np.ones(shape=self.out_op.shape[1].value), dtype=tf.float32)
        self.out_act = self.out_op + tf.random_normal(tf.shape(self.out_op), dtype=tf.float32) * self.std
        
        self.log_probs = gaussian_likelihood(self.act_holders, self.out_op, self.std)
        
        self.advantages = self.reward_holders - tf.squeeze(self.v_out_op)
        
        # Creation of ICM module
        
        self._create_ICM()
        
        self.r_e = tf.reduce_mean(self.log_probs * self.advantages)
        self.r_t = tf.reduce_mean(self.r_e + self.r_i) # Maybe I should reduce_sum or split by rollout
        
        self.lamb = 0.1
        self.beta = 0.2
        
        self.total_loss = -self.lamb * self.r_t + (1.-self.beta) * self.loss_i + self.beta * self.loss_f
        
        self.optimizer = optimizer
        self.actor_update = self.optimizer.minimize(self.total_loss)
        
        with tf.control_dependencies([self.actor_update]):
            self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - tf.squeeze(self.v_out_op)))
            self.value_update = self.optimizer.minimize(self.value_loss)
        
        def update_func(train_data):
            i_reward, _, _ = self.sess.run([self.r_i, self.actor_update, self.value_update], 
                               feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                    self.act_holders: reshape_train_var(train_data[:, 1]),
                                    self.reward_holders: train_data[:, 2],
                                    self.next_obs_holders: reshape_train_var(train_data[:, 3])})
            
            print(f'Intrinsic Reward: {np.mean(i_reward)}')    
        
        self.sess.run(tf.global_variables_initializer())
        
        return update_func
        
    def _gen_discrete_act(self, obs):
        act_probs = self.sess.run(self.out_op, feed_dict={self.in_op: [obs]})
        act = np.random.choice(list(range(len(act_probs[0]))), p=act_probs)
        
        return act
    
    def _gen_continuous_act(self, obs):
        act_vect = self.sess.run(self.out_act, feed_dict={self.in_op: [obs]})[0]
        
        return np.array(act_vect)
        
    def gen_act(self, obs):
        if self.act_type == 'discrete':
            return self._gen_discrete_act(obs)
        else:
            return self._gen_continuous_act(obs)
        
    def train(self, obs, rewards, acts):
        raise RuntimeError('The train method was not properly created')

In [3]:
env = gym.make('BipedalWalker-v2')

obs = tf.placeholder(tf.float32, shape=[None]+list(env.observation_space.shape))
dense1 = dense(obs, 32, activation=tf.tanh)
dense2 = dense(dense1, 32, activation=tf.tanh)
act_probs = dense(dense2, env.action_space.shape[0])

v_dense1 = dense(obs, 32, activation=tf.tanh)
v_dense2 = dense(v_dense1, 32, activation=tf.tanh)
value = dense(v_dense2, 1)

network = IMTrainer(obs, act_probs, value, act_type='c')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [4]:
n_episodes = 1000000
max_steps = 500
update_freq = 1
print_freq = 1

mb = MemoryBuffer(True)

In [ ]:
all_rewards = []

for episode in range(n_episodes):
    ep_reward = 0
    
    mb.start_rollout()
    obs = env.reset()
    for step in range(max_steps):
        obs = obs.squeeze()
        act = network.gen_act(obs)
        
        obs_next, rew, d, _ = env.step(act)
        ep_reward += rew
        
        if False:
            env.render()
            time.sleep(0.02)
        
        mb.record(obs, act, rew, obs_next)
        obs = obs_next
        
        if d:
            break
            
    all_rewards.append(ep_reward)
            
    if episode % update_freq == 0 and episode != 0:
        network.train(mb.to_data())
        
        if episode % (update_freq * print_freq) == 0:
            print(f'Update #{episode // update_freq}, Reward: {np.mean(all_rewards[-update_freq*print_freq:])}')
            print()

Intrinsic Reward: 141.8203582763672
Update #1, Reward: -108.1877613371412

Intrinsic Reward: 142.29800415039062
Update #2, Reward: -104.93920621598326

Intrinsic Reward: 141.77227783203125
Update #3, Reward: -106.67962389740099

Intrinsic Reward: 141.9000701904297
Update #4, Reward: -104.33232116363818

Intrinsic Reward: 140.94427490234375
Update #5, Reward: -107.94131572361849

Intrinsic Reward: 142.08303833007812
Update #6, Reward: -105.70714224652387

Intrinsic Reward: 142.25411987304688
Update #7, Reward: -103.10124309706129

Intrinsic Reward: 142.13668823242188
Update #8, Reward: -105.26173562503979

Intrinsic Reward: 141.6094970703125
Update #9, Reward: -103.0819227867201

Intrinsic Reward: 141.536376953125
Update #10, Reward: -105.80744348400646

Intrinsic Reward: 141.97628784179688
Update #11, Reward: -106.28764034203999

Intrinsic Reward: 141.01806640625
Update #12, Reward: -104.41805487677144

Intrinsic Reward: 141.51329040527344
Update #13, Reward: -104.37899782687549

Intri

Intrinsic Reward: 141.72335815429688
Update #110, Reward: -104.26009215658965

Intrinsic Reward: 141.6574249267578
Update #111, Reward: -107.90752240361336

Intrinsic Reward: 141.63916015625
Update #112, Reward: -106.99899438278501

Intrinsic Reward: 141.7860565185547
Update #113, Reward: -110.31637532760824

Intrinsic Reward: 142.11727905273438
Update #114, Reward: -41.71348057895774

Intrinsic Reward: 141.72140502929688
Update #115, Reward: -104.29485520391228

Intrinsic Reward: 141.54994201660156
Update #116, Reward: -108.84585291585388

Intrinsic Reward: 142.66180419921875
Update #117, Reward: -103.9837869064454

Intrinsic Reward: 141.74974060058594
Update #118, Reward: -106.5858596423405

Intrinsic Reward: 142.6212158203125
Update #119, Reward: -106.89436623358105

Intrinsic Reward: 141.53305053710938
Update #120, Reward: -108.18674954947085

Intrinsic Reward: 142.24517822265625
Update #121, Reward: -104.17497381555847

Intrinsic Reward: 142.01869201660156
Update #122, Reward: -10

Intrinsic Reward: 142.32594299316406
Update #215, Reward: -105.2364306797826

Intrinsic Reward: 143.23440551757812
Update #216, Reward: -105.94360520272826

Intrinsic Reward: 142.1772918701172
Update #217, Reward: -108.86083310614278

Intrinsic Reward: 142.567138671875
Update #218, Reward: -104.47535727840476

Intrinsic Reward: 142.13746643066406
Update #219, Reward: -107.39338784700011

Intrinsic Reward: 142.26535034179688
Update #220, Reward: -104.25057206912153

Intrinsic Reward: 142.28518676757812
Update #221, Reward: -109.19552208778511

Intrinsic Reward: 141.86550903320312
Update #222, Reward: -105.12299286158259

Intrinsic Reward: 142.04820251464844
Update #223, Reward: -104.3727084690885

Intrinsic Reward: 142.39585876464844
Update #224, Reward: -104.62331975010845

Intrinsic Reward: 142.29092407226562
Update #225, Reward: -104.39879182992317

Intrinsic Reward: 142.0066375732422
Update #226, Reward: -102.86811793591703

Intrinsic Reward: 142.55845642089844
Update #227, Reward: 

Intrinsic Reward: 142.36036682128906
Update #322, Reward: -105.89569049177692

Intrinsic Reward: 142.74835205078125
Update #323, Reward: -106.32280151266356

Intrinsic Reward: 142.4604949951172
Update #324, Reward: -103.83930251153558

Intrinsic Reward: 141.96853637695312
Update #325, Reward: -104.41018418573216

Intrinsic Reward: 142.9966278076172
Update #326, Reward: -106.32921150718629

Intrinsic Reward: 142.3785400390625
Update #327, Reward: -104.83515258962537

Intrinsic Reward: 141.86151123046875
Update #328, Reward: -105.18164910485285

Intrinsic Reward: 141.79034423828125
Update #329, Reward: -104.30367566339423

Intrinsic Reward: 142.29542541503906
Update #330, Reward: -105.07885783997179

Intrinsic Reward: 142.53541564941406
Update #331, Reward: -123.33502599776287

Intrinsic Reward: 143.22274780273438
Update #332, Reward: -105.72091999928219

Intrinsic Reward: 141.75975036621094
Update #333, Reward: -103.92336107873669

Intrinsic Reward: 142.54786682128906
Update #334, Rewar

Intrinsic Reward: 142.3104248046875
Update #430, Reward: -104.5389522219915

Intrinsic Reward: 142.68222045898438
Update #431, Reward: -104.6819708557104

Intrinsic Reward: 142.46730041503906
Update #432, Reward: -104.98617675299694

Intrinsic Reward: 142.92706298828125
Update #433, Reward: -107.05343660003754

Intrinsic Reward: 143.05465698242188
Update #434, Reward: -109.65999183366138

Intrinsic Reward: 142.15997314453125
Update #435, Reward: -102.62231514054474

Intrinsic Reward: 143.16653442382812
Update #436, Reward: -105.17222459227158

Intrinsic Reward: 142.51194763183594
Update #437, Reward: -105.39322349084914

Intrinsic Reward: 143.46444702148438
Update #438, Reward: -125.00921107586225

Intrinsic Reward: 143.5236053466797
Update #439, Reward: -103.60572553915097

Intrinsic Reward: 143.19671630859375
Update #440, Reward: -104.1702363601271

Intrinsic Reward: 142.39376831054688
Update #441, Reward: -104.63600510488327

Intrinsic Reward: 142.91026306152344
Update #442, Reward:

...

Intrinsic Reward: 143.96347045898438
Update #6984, Reward: -112.5681774665018

Intrinsic Reward: 143.96812438964844
Update #6985, Reward: -114.07052107878837

Intrinsic Reward: 143.9683074951172
Update #6986, Reward: -108.31109932731837

Intrinsic Reward: 143.97164916992188
Update #6987, Reward: -112.79015142055663

Intrinsic Reward: 143.974365234375
Update #6988, Reward: -113.95180262366559

Intrinsic Reward: 143.95423889160156
Update #6989, Reward: -116.76034346547598

Intrinsic Reward: 143.9683074951172
Update #6990, Reward: -111.26712359449206

Intrinsic Reward: 143.96231079101562
Update #6991, Reward: -112.53207347230365

Intrinsic Reward: 143.97262573242188
Update #6992, Reward: -112.84984318228376

Intrinsic Reward: 143.739501953125
Update #6993, Reward: -123.81473748485494

Intrinsic Reward: 143.840087890625
Update #6994, Reward: -124.93244494234212

Intrinsic Reward: 143.87013244628906
Update #6995, Reward: -125.95609270722048

Intrinsic Reward: 143.9682159423828
Update #6996,

Intrinsic Reward: 143.96559143066406
Update #7091, Reward: -109.30671788942877

Intrinsic Reward: 143.9582061767578
Update #7092, Reward: -116.41488343334757

Intrinsic Reward: 143.7359619140625
Update #7093, Reward: -122.5303349970368

Intrinsic Reward: 143.97386169433594
Update #7094, Reward: -111.8300633850079

Intrinsic Reward: 143.8599395751953
Update #7095, Reward: -124.4756243723159

Intrinsic Reward: 143.9512481689453
Update #7096, Reward: -112.51389477600965

Intrinsic Reward: 143.9720916748047
Update #7097, Reward: -111.2714380072821

Intrinsic Reward: 143.96847534179688
Update #7098, Reward: -115.50842731622978

Intrinsic Reward: 143.96673583984375
Update #7099, Reward: -114.1602826138232

Intrinsic Reward: 143.968017578125
Update #7100, Reward: -112.37806679434702

Intrinsic Reward: 143.91212463378906
Update #7101, Reward: -119.7321395948548

Intrinsic Reward: 143.72267150878906
Update #7102, Reward: -121.40897191951301

Intrinsic Reward: 143.72698974609375
Update #7103, Re

Intrinsic Reward: 143.96693420410156
Update #7195, Reward: -109.57410445786702

Intrinsic Reward: 143.93109130859375
Update #7196, Reward: -108.69511297274133

Intrinsic Reward: 143.9631805419922
Update #7197, Reward: -108.31491803688309

Intrinsic Reward: 143.9654998779297
Update #7198, Reward: -109.48502217018418

Intrinsic Reward: 143.9501190185547
Update #7199, Reward: -106.89441384964995

Intrinsic Reward: 143.96603393554688
Update #7200, Reward: -108.91524778682118

Intrinsic Reward: 143.9632568359375
Update #7201, Reward: -109.27193641656886

Intrinsic Reward: 143.96949768066406
Update #7202, Reward: -111.15152369780155

Intrinsic Reward: 143.92352294921875
Update #7203, Reward: -105.44912621930862

Intrinsic Reward: 143.9114990234375
Update #7204, Reward: -108.48607630888745

Intrinsic Reward: 143.9170684814453
Update #7205, Reward: -106.406840073814

Intrinsic Reward: 143.95494079589844
Update #7206, Reward: -107.72054993415438

Intrinsic Reward: 143.9390411376953
Update #7207

Intrinsic Reward: 143.9354705810547
Update #7299, Reward: -105.94002119604187

Intrinsic Reward: 143.96351623535156
Update #7300, Reward: -111.77993848585959

Intrinsic Reward: 143.9591827392578
Update #7301, Reward: -106.69854275671827

Intrinsic Reward: 143.93011474609375
Update #7302, Reward: -105.3355582019066

Intrinsic Reward: 143.9670867919922
Update #7303, Reward: -113.42967763788253

Intrinsic Reward: 143.95895385742188
Update #7304, Reward: -109.4464072782925

Intrinsic Reward: 143.9672088623047
Update #7305, Reward: -110.388974641166

Intrinsic Reward: 143.9689178466797
Update #7306, Reward: -111.60504936045098

Intrinsic Reward: 143.96139526367188
Update #7307, Reward: -109.04765046746036

Intrinsic Reward: 143.97293090820312
Update #7308, Reward: -112.1584969421855

Intrinsic Reward: 143.90640258789062
Update #7309, Reward: -120.58184030883262

Intrinsic Reward: 143.95526123046875
Update #7310, Reward: -106.52669854818222

Intrinsic Reward: 143.969482421875
Update #7311, R

Intrinsic Reward: 143.96873474121094
Update #7403, Reward: -111.34834616835664

Intrinsic Reward: 143.95245361328125
Update #7404, Reward: -110.39237231368323

Intrinsic Reward: 143.9400177001953
Update #7405, Reward: -108.38982498590151

Intrinsic Reward: 143.96055603027344
Update #7406, Reward: -110.53148399184272

Intrinsic Reward: 143.96864318847656
Update #7407, Reward: -109.92790959369019

Intrinsic Reward: 143.9664764404297
Update #7408, Reward: -108.76114884335858

Intrinsic Reward: 143.9467010498047
Update #7409, Reward: -106.54706718102781

Intrinsic Reward: 143.9485321044922
Update #7410, Reward: -111.64958847695216

Intrinsic Reward: 143.9714813232422
Update #7411, Reward: -109.54305774916274

Intrinsic Reward: 143.95413208007812
Update #7412, Reward: -113.98269039696144

Intrinsic Reward: 143.95867919921875
Update #7413, Reward: -109.81445521587133

Intrinsic Reward: 143.9410400390625
Update #7414, Reward: -117.52743708526157

Intrinsic Reward: 143.96640014648438
Update #7